In [3]:
import numpy as np
import pandas as pd

# plot
import matplotlib.pyplot as plt

# tf
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

import splitfolders

## Data Collecting

Dataset diambil dari website kaggle.

### Tujuan PA

Membuat sebuah model yang dapat mengenali jenis dari serangga dengan menggunakan algoritma CNN.

Batasan:
  - Model ini hanya dapat mengenali:
    - Kupu - Kupu, 
    - Capung, 
    - Nyamuk, 
    - Belalang,
    - dan Kumbang.

### Deskripsi Dataset

Dataset ini berupa gambar dari 5 serangga berbeda.

### Jumlah Dataset

- Kupu - Kupu: 899
- Capung: 1036
- Nyamuk: 690
- Belalang: 960
- Kumbang: 864

Total: 4449 data

### Data Train & Validation

URL path ke direktori dataset.
berupa direktori dataset.
dan dataset output (dataset yang akan dihasilkan ketika melakukan split folder). 

In [4]:
url = 'dataset_serangga'
url_split = 'dataset_output'

In [5]:
splitfolders.ratio(url, output=url_split, seed=42, ratio=(0.7,0.2,0.1), group_prefix=None)

Disini saya menggunakan Image data generator dan batch size sebanyak 128
untuk validation, saya menggunakan 20% data.
Kemudian untuk subset training dan validation, target size nya adalah 120.

ambil path dari folder masing - masing output

In [6]:
train = 'dataset_output/train'
val = 'dataset_output/val'
test = 'dataset_output/test'

In [7]:
batch_size = 128

train_image_generator = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=45,
    horizontal_flip=True,
) #data preprocessing

train = train_image_generator.flow_from_directory(
    train, 
    target_size=(120,120),
    batch_size=batch_size, 
    color_mode="grayscale"
)

test = train_image_generator.flow_from_directory(
    test, 
    target_size=(120,120),
    batch_size=batch_size,
    color_mode="grayscale"
)

val = train_image_generator.flow_from_directory(
    val, 
    target_size=(120,120),
    batch_size=batch_size,
    color_mode="grayscale"
)

Found 3112 images belonging to 5 classes.
Found 449 images belonging to 5 classes.
Found 888 images belonging to 5 classes.


# Model 

Pada syntax di atas model dibentuk dengan suatu layer convo 2 dimensi dari size 120 x 120 pixels dimana fungsi aktivasinya adalah. Terdapat dua pembuatan layer convo, sehingga dilanjutkan dengan pembuatan layer max pooling dan pembuatan layer dropout. Karena ini adalah pelatihan deep learning maka perlu pelatihan lagi dengan membuat lagi layer untuk pelatihan. Kemudian dilanjutkan dengan layer flatten dan layer dense sampai dengan proses kompilasi.



In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(
        32, 
        (3,3), 
        activation='relu',
        input_shape=(120, 120, 1), #parameter 3 untuk warna, kalau grayscale nilainya 1 kalau berwarna 3 
        ),
    tf.keras.layers.MaxPooling2D(2, 2), # untuk mengecilkan feature map

    tf.keras.layers.Conv2D(
      32,
      (3,3), 
      activation='relu',
      ),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(
        32, 
        (3,3), 
        activation='relu', 
        ),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    tf.keras.layers.Conv2D(
      128,
      (3,3), 
      activation='relu',
    ),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Dropout(0.3), # drop neuron secara random

    tf.keras.layers.Flatten(), # untuk jadikan 1dimensi

    tf.keras.layers.Dense(128, activation='relu'), 
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(5, activation='softmax'), #multiclass pakai softmax
])

model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# adam untuk mengurangi overfitting


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 118, 118, 32)      320       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 59, 59, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 57, 57, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 28, 28, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 26, 26, 32)        9248      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 13, 13, 32)       0

epoch dibuat sebanyak 30

In [11]:
# coba pakai callback custom misalnya: stop kalau accuracy train > 95%, dan earlystopping
history = model.fit(train, validation_data=val, epochs=30)

Epoch 1/30
25/25 [==============================] - 66s 2s/step - loss: 1.5950 - accuracy: 0.2314 - val_loss: 1.5569 - val_accuracy: 0.3559
Epoch 2/30
25/25 [==============================] - 54s 2s/step - loss: 1.5325 - accuracy: 0.3294 - val_loss: 1.4573 - val_accuracy: 0.3592
Epoch 3/30
25/25 [==============================] - 48s 2s/step - loss: 1.4736 - accuracy: 0.3535 - val_loss: 1.4017 - val_accuracy: 0.4009
Epoch 4/30
25/25 [==============================] - 53s 2s/step - loss: 1.4188 - accuracy: 0.3911 - val_loss: 1.3797 - val_accuracy: 0.4167
Epoch 5/30
25/25 [==============================] - 54s 2s/step - loss: 1.3771 - accuracy: 0.4103 - val_loss: 1.3378 - val_accuracy: 0.4505
Epoch 6/30
 6/25 [======>.......................] - ETA: 22s - loss: 1.3739 - accuracy: 0.4245

In [ ]:
model.evaluate(test)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Akurasi Training')
plt.plot(epochs, val_acc, 'g', label='Akurasi Validation')

plt.xlabel('Epochs')
plt.ylabel('Akurasi')
plt.title('Akurasi Training and validation')
plt.grid(axis='both')

plt.show() 